
# トヨタ株 (7203.T) — 翌日終値予測 + 売買判断（Buy/Sell）
**目的**: 2020/09〜2025/09 の5年間の株価データを用い、  
1) 翌日の終値を**回帰**で予測し、  
2) その予測に基づいて**売買判断（Buy/Sell）**の**分類**を行う。

**ポイント**  
- クリプト向けチュートリアル（15分足 / GMO Fetcher）を **日次株価 / Yahoo Finance 取得**に置換。  
- `GmoFetcher` 相当の薄いラッパー (`YfFetcher`) を実装し、**キャッシュ（joblib.Memory）**で再取得を抑制。  
- **LSTM** を用いた時系列ウィンドウ学習（回帰）＋ 予測結果を使った **2値分類**。  
- ベースライン（線形回帰）と比較、さらに**指標可視化**、**単純バックテスト**を実装。

> 環境にネットアクセスが無い場合は、Kaggle等からダウンロードした `7203.T.csv` を読み込むパスも提供します。


In [24]:

# === Imports ===
import os
import sys
import math
import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import joblib
from joblib import Memory
from pathlib import Path
from datetime import datetime

# Plot
import matplotlib.pyplot as plt

# ML/DL
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import TimeSeriesSplit

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Optional: yfinance for data download (requires internet)
try:
    import yfinance as yf
    HAS_YF = True
except Exception:
    HAS_YF = False

print('TensorFlow:', tf.__version__)
print('HAS_YF:', HAS_YF)


TensorFlow: 2.20.0
HAS_YF: True


In [25]:
# === Parameters ===
TICKER = "7203.T"  # トヨタ自動車
START_DATE = "2020-09-01"
END_DATE   = "2025-09-30"

CACHE_DIR = Path('/tmp/yf_cache_v2')
CACHE_DIR.mkdir(parents=True, exist_ok=True)
memory = Memory(location=str(CACHE_DIR), verbose=0)

# 学習ハイパラ
WINDOW_SIZE = 30     # 何日分の履歴で翌日を予測するか
BATCH_SIZE  = 64
EPOCHS      = 50
VAL_SPLIT   = 0.0     # 明示的に時系列分割するので 0

# 時系列分割（固定境界）
SPLIT_TRAIN_END = "2024-03-31"
SPLIT_VAL_END   = "2025-03-31"  # val: 2024-04-01〜2025-03-31
# test: 2025-04-01〜2025-09-30

# 乱数シード
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [28]:
# === YfFetcher: GmoFetcher 相当の簡易ラッパ ===
def _download_ohlcv_v2(ticker, start, end, interval='1d', csv_path=None):
    if csv_path is not None and Path(csv_path).exists():
        df = pd.read_csv(csv_path, parse_dates=['Date'])
        df = df.set_index('Date')
        return df

    if not HAS_YF:
        raise RuntimeError("yfinance が利用できません。csv_path を指定してください。")

    return yf.download(ticker, start=start, end=end, interval=interval, progress=False)


class YfFetcher:
    def __init__(self, memory=None):
        self.memory = memory
        if memory is not None:
            # メモリキャッシュは関数定義が変わると壊れるのでバージョン付きで保持
            self._fetch_fn = memory.cache(_download_ohlcv_v2)
        else:
            self._fetch_fn = _download_ohlcv_v2

    @staticmethod
    def _normalize_df(df):
        # yfinance の列名を統一し、DatetimeIndexへ
        # (Open High Low Close Adj Close Volume)
        if df is None or len(df) == 0:
            return pd.DataFrame()
        df = df.copy()
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        if hasattr(df.columns, 'nlevels') and df.columns.nlevels > 1:
            df.columns = [col if isinstance(col, str) else col[0] for col in df.columns]
        # 列名を小文字に揃える
        cols = {c: c.lower().replace(' ', '') for c in df.columns}
        df = df.rename(columns=cols)
        # 必須列があるかチェック
        must = ['open', 'high', 'low', 'close', 'adjclose', 'volume']
        for m in must:
            if m not in df.columns:
                # adjclose がないケースもあるので Close を複製
                if m == 'adjclose' and 'close' in df.columns:
                    df['adjclose'] = df['close']
                else:
                    raise ValueError(f"Missing column: {m}")
        return df[['open','high','low','close','adjclose','volume']]

    def fetch_ohlcv(self, ticker, start, end, interval='1d', csv_path=None):
        """
        ticker: 例 '7203.T'
        start, end: 'YYYY-MM-DD'
        interval: '1d' 固定（株は日次で扱う）
        csv_path: ローカルCSVのパス（ネット不可時）
        """
        raw_df = self._fetch_fn(ticker, start, end, interval, csv_path)
        return self._normalize_df(raw_df)


fetcher = YfFetcher(memory=memory)


In [ ]:

# === データ取得 ===
# ネット不可なら csv_path を指定して利用: e.g., './7203.T.csv'
CSV_PATH = "data/TM_1980-01-01_2025-06-27.csv"

df = fetcher.fetch_ohlcv(
    ticker=TICKER,
    start=START_DATE,
    end=END_DATE,
    interval='1d',
    csv_path=CSV_PATH
)

print(df.head())
print(df.tail())
print(df.describe())

# # 保存（チュートリアル互換）
df.to_pickle('df_ohlcv_7203T.pkl')


AttributeError: 'tuple' object has no attribute 'lower'

In [29]:


# === 期間フィルタ（念のため） ===
if 'df' not in globals():
    fallback_path = Path('df_ohlcv_7203T.pkl')
    if fallback_path.exists():
        df = pd.read_pickle(fallback_path)
        print(f"Loaded cached OHLCV data from {fallback_path}")
    else:
        raise RuntimeError('価格データ(df)が存在しません。先にデータ取得セルを実行するか df_ohlcv_7203T.pkl を用意してください。')

df = df[(df.index >= pd.to_datetime(START_DATE)) & (df.index <= pd.to_datetime(END_DATE))].copy()

# === テクニカル指標・派生特徴量 ===
def rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ma_up = up.rolling(window=period, min_periods=period).mean()
    ma_down = down.rolling(window=period, min_periods=period).mean()
    rs = ma_up / (ma_down + 1e-9)
    return 100 - (100 / (1 + rs))

def macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def bollinger(series, window=20, num_std=2):
    ma = series.rolling(window=window, min_periods=window).mean()
    std = series.rolling(window=window, min_periods=window).std()
    upper = ma + num_std * std
    lower = ma - num_std * std
    width = (upper - lower) / (ma + 1e-9)
    return ma, upper, lower, width

# 主要な終値ベースで算出
close = df['close']

df['ret_1d'] = close.pct_change()
df['ma_7']   = close.rolling(7).mean()
df['ma_30']  = close.rolling(30).mean()
df['ema_7']  = close.ewm(span=7, adjust=False).mean()
df['ema_30'] = close.ewm(span=30, adjust=False).mean()
df['rsi_14'] = rsi(close, 14)

macd_line, signal_line, macd_hist = macd(close)
df['macd']   = macd_line
df['macd_s'] = signal_line
df['macd_h'] = macd_hist

bb_ma, bb_up, bb_lo, bb_w = bollinger(close, 20, 2)
df['bb_ma'] = bb_ma
df['bb_up'] = bb_up
df['bb_lo'] = bb_lo
df['bb_w']  = bb_w

# 出来高系
df['vol_chg'] = df['volume'].pct_change()

# 1日先の終値（回帰ターゲット）
df['target_close_t1'] = df['close'].shift(-1)

# 1日先が上昇なら1（Buy）、下降なら0（Sell）
df['target_buy'] = (df['target_close_t1'] > df['close']).astype(float)

# 欠損除去
df = df.dropna().copy()
print('Final shape with features:', df.shape)

df.head(3)


RuntimeError: 価格データ(df)が存在しません。先にデータ取得セルを実行するか df_ohlcv_7203T.pkl を用意してください。

In [ ]:

# === 可視化（概観） ===
fig, ax = plt.subplots(figsize=(12,4))
df['close'].plot(ax=ax)
ax.set_title('Toyota (7203.T) Close')
ax.grid(True)
plt.show()

fig, ax = plt.subplots(figsize=(12,3))
df['rsi_14'].plot(ax=ax)
ax.axhline(70, linestyle='--'); ax.axhline(30, linestyle='--')
ax.set_title('RSI(14)')
ax.grid(True)
plt.show()


In [ ]:

# === 時系列分割 ===
train = df[df.index <= SPLIT_TRAIN_END].copy()
val   = df[(df.index > SPLIT_TRAIN_END) & (df.index <= SPLIT_VAL_END)].copy()
test  = df[df.index > SPLIT_VAL_END].copy()

print('train:', train.index.min(), '->', train.index.max(), len(train))
print('val  :', val.index.min(),   '->', val.index.max(),   len(val))
print('test :', test.index.min(),  '->', test.index.max(),  len(test))

# 特徴量カラム
FEATURE_COLS = [
    'open','high','low','close','adjclose','volume',
    'ret_1d','ma_7','ma_30','ema_7','ema_30','rsi_14',
    'macd','macd_s','macd_h','bb_ma','bb_up','bb_lo','bb_w','vol_chg'
]

def make_window_dataset(df_part, feature_cols, target_col, window):
    X_list, y_list = [], []
    feats = df_part[feature_cols].values
    target = df_part[target_col].values
    for i in range(len(df_part) - window):
        X_list.append(feats[i:i+window])
        y_list.append(target[i+window])
    return np.array(X_list), np.array(y_list)

# スケーラーは訓練でfitし、他でtransform
scaler = StandardScaler()
scaler.fit(train[FEATURE_COLS].values)

def scale_df(df_part):
    cp = df_part.copy()
    cp[FEATURE_COLS] = scaler.transform(cp[FEATURE_COLS].values)
    return cp

train_s = scale_df(train)
val_s   = scale_df(val)
test_s  = scale_df(test)

# 回帰用データセット（翌日終値）
X_train_reg, y_train_reg = make_window_dataset(train_s, FEATURE_COLS, 'target_close_t1', WINDOW_SIZE)
X_val_reg,   y_val_reg   = make_window_dataset(val_s,   FEATURE_COLS, 'target_close_t1', WINDOW_SIZE)
X_test_reg,  y_test_reg  = make_window_dataset(test_s,  FEATURE_COLS, 'target_close_t1', WINDOW_SIZE)

# 分類用（Buy/Sell）
X_train_cls, y_train_cls = make_window_dataset(train_s, FEATURE_COLS, 'target_buy', WINDOW_SIZE)
X_val_cls,   y_val_cls   = make_window_dataset(val_s,   FEATURE_COLS, 'target_buy', WINDOW_SIZE)
X_test_cls,  y_test_cls  = make_window_dataset(test_s,  FEATURE_COLS, 'target_buy', WINDOW_SIZE)

X_train_reg.shape, X_val_reg.shape, X_test_reg.shape


In [ ]:

# === ベースライン：線形回帰（回帰） ===
# ウィンドウを平均で潰して単純特徴に落とす簡易ベースライン
def collapse_window_mean(X):
    # (N, window, F) -> (N, F) by mean
    return X.mean(axis=1)

Xtr_bl = collapse_window_mean(X_train_reg)
Xv_bl  = collapse_window_mean(X_val_reg)
Xte_bl = collapse_window_mean(X_test_reg)

linr = LinearRegression()
linr.fit(Xtr_bl, y_train_reg)

pred_tr_bl = linr.predict(Xtr_bl)
pred_v_bl  = linr.predict(Xv_bl)
pred_te_bl = linr.predict(Xte_bl)

def rmse(y, p): return math.sqrt(mean_squared_error(y, p))

print('Baseline Linear Regression')
print('  Train RMSE:', rmse(y_train_reg, pred_tr_bl))
print('  Val   RMSE:', rmse(y_val_reg,   pred_v_bl))
print('  Test  RMSE:', rmse(y_test_reg,  pred_te_bl))

print('  Test R2  :', r2_score(y_test_reg, pred_te_bl))


In [ ]:

# === LSTM（回帰：翌日終値） ===
tf.keras.backend.clear_session()

model_reg = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_reg.shape[1], X_train_reg.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dense(1, activation='linear')
])
model_reg.compile(optimizer='adam', loss='mse')

callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
    ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', verbose=1),
    ModelCheckpoint('best_regression.keras', monitor='val_loss', save_best_only=True, verbose=0)
]

hist = model_reg.fit(
    X_train_reg, y_train_reg,
    validation_data=(X_val_reg, y_val_reg),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

# 推論
pred_tr = model_reg.predict(X_train_reg).ravel()
pred_v  = model_reg.predict(X_val_reg).ravel()
pred_te = model_reg.predict(X_test_reg).ravel()

print('LSTM Regression')
print('  Train RMSE:', rmse(y_train_reg, pred_tr))
print('  Val   RMSE:', rmse(y_val_reg,   pred_v))
print('  Test  RMSE:', rmse(y_test_reg,  pred_te))
print('  Test R2  :', r2_score(y_test_reg, pred_te))

# 学習曲線
plt.figure(figsize=(8,4))
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val')
plt.title('LSTM Regression Loss')
plt.legend(); plt.grid(True); plt.show()


In [ ]:

# === 分類：予測終値に基づく Buy(1)/Sell(0) 判定 ===
# 判定ロジック： pred_close_{t+1} > actual_close_{t} ? 1 : 0
# 時系列整合のため、各セットの基準 day_t の close 実値を準備する

def get_last_close_vector(df_part, window):
    # 各サンプルの "直近日の実Close" を取り出す (ラベル生成用)
    vals = df_part['close'].values  # *スケール前* が本来望ましいが、ここは y と比較するだけなので OK
    # ウィンドウで切った最後の行の index を対応づけ
    out = []
    for i in range(len(df_part) - window):
        out.append(vals[i + window - 1])
    return np.array(out)

close_train_tail = get_last_close_vector(train, WINDOW_SIZE)
close_val_tail   = get_last_close_vector(val,   WINDOW_SIZE)
close_test_tail  = get_last_close_vector(test,  WINDOW_SIZE)

buy_pred_tr = (pred_tr > close_train_tail).astype(int)
buy_pred_v  = (pred_v  > close_val_tail).astype(int)
buy_pred_te = (pred_te > close_test_tail).astype(int)

print('=== Classification Metrics (Buy=1 / Sell=0) ===')
def cls_metrics(y_true, y_pred):
    return {
        'acc': accuracy_score(y_true, y_pred),
        'prec': precision_score(y_true, y_pred, zero_division=0),
        'rec': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
    }

print('Train:', cls_metrics(y_train_cls, buy_pred_tr))
print('Val  :', cls_metrics(y_val_cls,   buy_pred_v))
print('Test :', cls_metrics(y_test_cls,  buy_pred_te))

# Confusion Matrix (Test)
cm = confusion_matrix(y_test_cls, buy_pred_te)
print('\nConfusion Matrix (Test)\n', cm)
print('\nClassification Report (Test)\n', classification_report(y_test_cls, buy_pred_te, zero_division=0))


In [ ]:

# === シンプル・バックテスト（テスト区間のみ） ===
# ルール：
#   Buy(1) -> 翌日寄りで買って翌日引けで手仕舞い（= 翌日終値と当日終値の差分に連動すると仮定）
#   Sell(0) -> 何もしない（空売り等は考慮しないシンプル版）

test_close = test['close'].values
# ウィンドウ切り詰めに合わせて test_close を末尾1日分削る（y_test_reg と長さ一致）
test_close_tail = test_close[WINDOW_SIZE-1: -1]  # day_t close
test_close_next = test_close[WINDOW_SIZE:]       # day_{t+1} close

# 収益率（Buyのときのみリターンを計上）
ret = np.zeros_like(buy_pred_te, dtype=float)
price_diff = (test_close_next - test_close_tail) / (test_close_tail + 1e-9)  # 日次騰落率
ret[buy_pred_te == 1] = price_diff[buy_pred_te == 1]

cum_ret = (1 + ret).cumprod() - 1

plt.figure(figsize=(10,4))
plt.plot(cum_ret, label='Strategy (Buy on predicted up)')
plt.axhline(0, color='k', linestyle='--')
plt.title('Cumulative Return (Test Period)')
plt.grid(True); plt.legend(); plt.show()

print('Final cumulative return (test): {:.2%}'.format(cum_ret[-1] if len(cum_ret)>0 else 0.0))


In [ ]:

# === 保存物 ===
joblib.dump(scaler, 'scaler_7203T.joblib')
model_reg.save('model_regression_7203T.keras')

with open('params_7203T.txt', 'w', encoding='utf-8') as f:
    f.write(f'Ticker: {TICKER}\n')
    f.write(f'Period: {START_DATE}..{END_DATE}\n')
    f.write(f'Window: {WINDOW_SIZE}\n')
    f.write(f'Train end: {SPLIT_TRAIN_END}\nVal end: {SPLIT_VAL_END}\n')
print('Artifacts saved.')



## メモ：クリプト向けチュートリアルからの置換ポイント
- **GmoFetcher → YfFetcher**  
  - 15分足 / `interval_sec` は株の**日次**に読み替え（`interval='1d'`）。  
  - `market='BTC_JPY'` → `ticker='7203.T'`。  
  - キャッシュは `joblib.Memory('/tmp/yf_cache')` を用意。

- **データ期間の限定**  
  - `df = df[df.index < '2021-04-01']` のような扱いを、`START_DATE`〜`END_DATE` で制御。

- **目的変数**  
  - 回帰：翌日終値 `target_close_t1`。  
  - 分類：`pred_close_{t+1} > close_t` を Buy=1 / Sell=0。

- **評価**  
  - 回帰：RMSE / R²。  
  - 分類：Accuracy / Precision / Recall / F1、Confusion Matrix。  
  - 簡易バックテストで実務的な感触も確認。

- **ネットワーク非依存**  
  - ネット不可の環境では Kaggle 等で `7203.T.csv` を持ち込み、`csv_path` に指定。
